# Example 23: Code Review Assistance

## Learning Objective
Learn to use Claude Code for thorough code reviews.

## Code to Review

In [ ]:
# This code has several issues - can you find them?
import json

def process_user_data(file_path):
    f = open(file_path)
    data = json.load(f)
    f.close()
    
    users = []
    for user in data:
        if user['age'] > 18:
            user['status'] = 'adult'
        else:
            user['status'] = 'minor'
        
        user['full_name'] = user['first_name'] + ' ' + user['last_name']
        
        if user['email'].find('@') == -1:
            print('Invalid email: ' + user['full_name'])
            continue
        
        users.append(user)
    
    output = open('processed.json', 'w')
    json.dump(users, output)
    output.close()
    
    return users

## Issues Found

### Critical Issues

1. **File not closed on error** - If `json.load()` fails, file stays open
2. **Missing KeyError handling** - What if 'first_name' is missing?

In [ ]:
# CRITICAL: File handle leak
# Bad:
f = open(file_path)
data = json.load(f)  # If this fails, f is never closed!
f.close()

# Good:
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)  # File automatically closed even on error

### Warnings

3. **Hardcoded output path** - Should be parameterized
4. **No encoding specified** - Should use `encoding='utf-8'`
5. **Using print() instead of logging**

In [ ]:
import logging
logger = logging.getLogger(__name__)

# Instead of print(), use logging:
logger.warning(f"Invalid email for user: {user['full_name']}")

### Suggestions

6. **Use f-strings** instead of concatenation
7. **Use `.get()` for safe dictionary access**
8. **Add type hints**

## Improved Version

In [ ]:
import json
import logging
from pathlib import Path

logger = logging.getLogger(__name__)


def process_user_data(
    input_path: str | Path,
    output_path: str | Path | None = None
) -> list[dict]:
    """Process user data from JSON file.
    
    Args:
        input_path: Path to input JSON file.
        output_path: Path for output file (optional).
    
    Returns:
        List of processed user dictionaries.
    """
    # Use context manager
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    users = []
    for user in data:
        # Safe dictionary access
        age = user.get('age', 0)
        user['status'] = 'adult' if age > 18 else 'minor'
        
        # f-string and safe access
        first = user.get('first_name', '')
        last = user.get('last_name', '')
        user['full_name'] = f"{first} {last}".strip()
        
        # Better email validation
        email = user.get('email', '')
        if '@' not in email:
            logger.warning(f"Invalid email for: {user['full_name']}")
            continue
        
        users.append(user)
    
    # Write output if path provided
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(users, f, indent=2)
    
    return users

print("Improved version defined!")

## Code Review Prompts

```
Review this code for:
1. Bugs and potential errors
2. Security issues
3. Performance problems
4. Best practices
5. Missing error handling
```

In [ ]:
# Practice: Review this code for security issues
def authenticate(username, password):
    query = "SELECT * FROM users WHERE username='" + username + "' AND password='" + password + "'"
    result = db.execute(query)
    return result is not None

# SECURITY ISSUE: SQL Injection vulnerability!
# Malicious input: username = "admin' --" could bypass authentication